In [1]:
# import Selenium components for scraping
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# import pandas to create dataframe
import pandas as pd

# import numpy to deal with NaN values
import numpy as np

# import datetime for consistent column naming
from datetime import date
from datetime import timedelta

# remove warning when changing score dtype to float
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# set up Chrome driver and direct to target brewery
# this needs to be the same brewery as in the create_df.ipynb file
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://untappd.com/login?go_to=https://untappd.com/BurialBeer/beer")

### Logging in

First we need to log in because Untappd won't allow us to show more than 25 beers without being logged in. I'm sure there are some complex solutions to getting around the image selector the the reCaptcha near the end of this login process, but for now this program is manually run anyway so we will need to finish the login process in the browser. I've included code to get us to the end of the reCaptcha mostly for fun and as a challenge to myself.

You will need to fill in your own username and password as strings in the variables below.

In [3]:
# login credentials
username = 'username'
password = 'password'

# log in
driver.find_element(By.ID, 'username').send_keys(username)
driver.find_element(By.ID, 'password').send_keys(password)

# reCaptcha is behind an iFrame
WebDriverWait(driver, 20).until(
    EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[@title='reCAPTCHA']"))
)
WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "div.recaptcha-checkbox-border"))
).click()

# reCAPTCHA image selector pops up, manually select and submit login

### Sorting beers and showing all

When we've successfully logged in, we can run the next step. We will sort the beers from old to new so that beers in our original list will always stay in the same position and the columns we add in the future, when joined, will just lengthen the list.

This step will end in a "Timeout Exception" error. When the "Show More" button is no longer visible this step will time out, however we will have gotten the result we want -- all of the brewery's beers will now be displayed on the page.

In [3]:
# sort beers from old to new
driver.get('https://untappd.com/BurialBeer/beer?sort=created_at')

# show all beers by clicking "Show More" button
show_more = driver.find_element(By.XPATH, '//a[@class="button yellow more-list-items track-click"]')
show_more_presence = driver.find_elements(By.XPATH, '//a[@class="button yellow more-list-items track-click"]')

while len(show_more_presence) > 0:
    show_more.click()
    WebDriverWait(driver, 5).until(
        EC.visibility_of_element_located((By.XPATH, '//a[@class="button yellow more-list-items track-click"]'))
    )

TimeoutException: Message: 


### Creating our lists and dataframe

In [4]:
# find beer names
beer_names = driver.find_elements(By.XPATH, '//p[@class="name"]')

# put beer name text in a list
beer_name_list = []
for name in range(len(beer_names)):
    beer_name_list.append(beer_names[name].text)

In [5]:
# find beer styles
beer_styles = driver.find_elements(By.XPATH, '//p[@class="style"]')

# put style text in list
style_list = []
for style in range(len(beer_styles)):
    style_list.append(beer_styles[style].text)
    
# skip overall brewery style (first)
style_list = style_list[1:]

In [6]:
# find beer scores
beer_scores = driver.find_elements(By.XPATH, '//span[@class="num"]')

# create list of beer scores
beer_score_list_unformatted = []
for score in range(len(beer_scores)):
    beer_score_list_unformatted.append(beer_scores[score].text)
    
# remove parentheses around scores
beer_score_list = []
for score in beer_score_list_unformatted:
    beer_score_list.append(score.strip('()'))

# skip overall brewery score (first)
beer_score_list = beer_score_list[1:]

In [7]:
# create dataframe
df = pd.DataFrame()

# use today's date to name our score column
today_str = date.today().strftime('%m-%d-%Y')

# add columns to dataframe
df['beer_names'] = beer_name_list
df['style'] = style_list
df[today_str] = beer_score_list

In [8]:
# check dataframe
df

,beer_names,style,07-30-2022
0,Hatchet,Pilsner - German,3.58
1,Skillet Donut Stout,Stout - Coffee,4.17
2,Billhook Farmhouse Ale,Farmhouse Ale - Saison,3.7
3,Scythe Rye IPA,IPA - Rye,3.92
4,Hacksaw Dunkel,Lager - Munich Dunkel,3.65
...,...,...,...
1229,Treading Onward Towards Oblivion,IPA - Imperial / Double,N/A
1230,Provincial Illusions of Antiquity,IPA - American,N/A
1231,Tonin' the Destroyer,IPA - American,4.18
1232,Please Rescue Me From My Own Ignorant Barrage ...,Stout - Imperial / Double,4.48


Currently our columns are both object types, so before we move on, lets change beers with N/A as their score to NaNs, change names to strings, and change scores to floats.

In [10]:
# replace 'N/A' scores as NaN
df[today_str].replace('N/A', np.nan, inplace=True)

# cast names as strings
df['beer_names'] = df['beer_names'].astype('string')

# cast styles as strings
df['style'] = df['style'].astype('string')

# cast scores as floats
df[today_str] = df[today_str].astype('float64')

# check to make sure dtypes changed successfully
df.dtypes

beer_names            string
style                 string
scores_07-28-2022    float64
dtype: object

In [11]:
df

,beer_names,style,scores_07-28-2022
0,Hatchet,Pilsner - German,3.58
1,Skillet Donut Stout,Stout - Coffee,4.17
2,Billhook Farmhouse Ale,Farmhouse Ale - Saison,3.70
3,Scythe Rye IPA,IPA - Rye,3.92
4,Hacksaw Dunkel,Lager - Munich Dunkel,3.65
...,...,...,...
1228,A Collection of Idiocy From A Top Notch Moron,IPA - Imperial / Double,NaN
1229,Treading Onward Towards Oblivion,IPA - Imperial / Double,NaN
1230,Provincial Illusions of Antiquity,IPA - American,NaN
1231,Tonin' the Destroyer,IPA - American,4.24


In [ ]:
# create variable to hold yesterday's date for opening most recent file
today = date.today()
yesterday = (today-timedelta(days=1)).strftime('%m-%d-%Y')

# import original dataframe to join with
# use index_col to prevent 'Unnamed: 0' column in join
last = pd.read_csv('burial_untappd_'+yesterday+'.csv', index_col=[0])

In [ ]:
# full/outer join with new scores
new_df = pd.merge(last, df, on=['beer_names', 'style'], how='outer')

In [ ]:
# check the joined dataframe
new_df

In [ ]:
# save new dataframe with date
new_df.to_csv('burial_untappd_'+today_str+'.csv')